In [6]:
using Plots
gr()

using TensorFlow

## Problem 2 from https://arxiv.org/pdf/physics/9705023.pdf


$$\dfrac{du}{dt}+\dfrac{1}{5}u=e^{-\frac{t}{5}}cos(t)$$
 

With $u(0)=0$
  and $t\in[0,2]$
 

Analytic solution is $$u_{ana}(t)=e^{-\frac{t}{5}}sin(t)$$
 

Trail solution is $$u_{trial}(t)=tN(t,\tilde{p})$$
 

In [7]:

sess=Session(Graph())
hl_width=64
a=10
@tf begin #Automatically name nodes based on RHS
    t = placeholder(Float32; shape=[-1])
    tt = expand_dims(t, 2) #make it a matrix
    
    w1 = get_variable([1,hl_width], Float32)
    b1 = get_variable([hl_width], Float32)
    w2 = get_variable([hl_width,1], Float32)
    
    #$u_{trial} = t*N(t, p̃). p̃={w1, b1, w2}
    net1 = nn.sigmoid(tt*w1 + b1)*w2
    u = tt.*net1
    
    
    #problem defn: du/dt = a*u(t)
    du_dt = gradients(u, tt)
    deq_lhs = du_dt + u/5
    deq_rhs = exp(-tt/5).*cos(tt)
        
    
    loss = reduce_sum((deq_lhs - deq_rhs).^2)
    opt = train.minimize(train.AdamOptimizer(), loss)
end


<Tensor Group:1 shape=unknown dtype=Any>

In [8]:
t_obs = collect(0.0:0.02:2.0)

run(sess, global_variables_initializer())
for ii in 1:1000
    _, loss_o, _, _ = run(sess, [opt, loss, deq_lhs, deq_rhs], Dict(t=>t_obs))
    if ii%50 == 1
        println(loss_o)
    end
end

33.036148
19.072584
17.523647
13.570286
6.552186
1.2517709
0.10162182
0.054201182
0.05375421
0.053502183
0.053237207
0.052960213
0.052672334
0.05237432
0.052067187
0.05175192
0.05142917
0.051099855
0.050764732
0.050424673


In [9]:
u_net = run(sess, u, Dict(t=>t_obs))
u_ana = exp(-t_obs/5).*sin(t_obs)


plot(t_obs, [u_net u_ana]; label=["u_net", "u_ana"])


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,384.952 36.2583,374.047 41.9129,363.303 47.5675,352.72 53.222,342.297 58.8766,332.036 64.5312,321.935 70.1858,311.996 75.8404,302.217 81.495,292.6 
 87.1496,283.143 92.8042,273.848 98.4588,264.714 104.113,255.74 109.768,246.928 115.423,238.277 121.077,229.787 126.732,221.459 132.386,213.291 138.041,205.284 
 143.696,197.437 149.35,189.752 155.005,182.227 160.659,174.864 166.314,167.66 171.969,160.617 177.623,153.735 183.278,147.013 188.932,140.451 194.587,134.049 
 200.241,127.807 205.896,121.725 211.551,115.802 217.205,110.039 222.86,104.435 228.514,98.9909 234.169,93.7055 239.824,88.5789 245.478,83.6109 251.133,78.8014 
 256.787,74.1502 262.442,69.657 268.097,65.3217 273.751,61.1441 279.406,57.1238 285.06,53.2608 290.715,49.5546 296.37,46.0052 302.024,42.6122 307.679,39.3753 
 313.333,36.2944 318.988,33.3691 324.643,30.599 330.297,27.9841 335.952,25.524 341.606,23.2184 347.261,21.0668 352.915,19.0691 358.57,17.225 364.225,15.5341 
 369.879,13.996 375.534,12.6105 381.188,11.3771 386.843,10.2956 392.498,9.36548 398.152,8.58655 403.807,7.95835 409.461,7.48055 415.116,7.15259 420.771,6.97433 
 426.425,6.94532 432.08,7.06493 437.734,7.33314 443.389,7.74918 449.044,8.31285 454.698,9.02363 460.353,9.88117 466.007,10.885 471.662,12.0346 477.317,13.3296 
 482.971,14.7696 488.626,16.3541 494.28,18.0826 499.935,19.9547 505.59,21.9698 511.244,24.1277 516.899,26.4275 522.553,28.8691 528.208,31.4519 533.862,34.1753 
 539.517,37.0389 545.172,40.0422 550.826,43.1846 556.481,46.4656 562.135,49.8848 567.79,53.4416 573.445,57.1355 579.099,60.9659 584.754,64.9323 590.408,69.0342 
 596.063,73.271 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,384.952 36.2583,374.765 41.9129,364.664 47.5675,354.652 53.222,344.732 58.8766,334.908 64.5312,325.183 70.1858,315.56 75.8404,306.043 81.495,296.634 
 87.1496,287.336 92.8042,278.153 98.4588,269.088 104.113,260.143 109.768,251.321 115.423,242.624 121.077,234.056 126.732,225.618 132.386,217.314 138.041,209.145 
 143.696,201.115 149.35,193.224 155.005,185.476 160.659,177.872 166.314,170.415 171.969,163.106 177.623,155.947 183.278,148.94 188.932,142.086 194.587,135.388 
 200.241,128.846 205.896,122.463 211.551,116.24 217.205,110.177 222.86,104.277 228.514,98.5394 234.169,92.9666 239.824,87.5592 245.478,82.3181 251.133,77.2441 
 256.787,72.3381 262.442,67.6006 268.097,63.0324 273.751,58.634 279.406,54.4059 285.06,50.3483 290.715,46.4618 296.37,42.7464 302.024,39.2025 307.679,35.83 
 313.333,32.6291 318.988,29.5996 324.643,26.7415 330.297,24.0546 335.952,21.5387 341.606,19.1933 347.261,17.0182 352.915,15.0129 358.57,13.1768 364.225,11.5094 
 369.879,10.01 375.534,8.6779 381.188,7.51231 386.843,6.51239 392.498,5.67724 398.152,5.00588 403.807,4.49729 409.461,4.15039 415.116,3.96403 420.771,3.93701 
 426.425,4.06806 432.08,4.35589 437.734,4.79911 443.389,5.3963 449.044,6.146 454.698,7.04667 460.353,8.09674 466.007,9.29459 471.662,10.6385 477.317,12.1269 
 482.971,13.7578 488.626,15.5295 494.28,17.4402 499.935,19.4879 505.59,21.6708 511.244,23.9867 516.899,26.4338 522.553,29.0099 528.208,31.7129 533.862,34.5408 
 539.517,37.4913 545.172,40.5623 550.826,43.7515 556.481,47.0567 562.135,50.4755 567.79,54.0056 573.445,57.6448 579.099,61.3905 584.754,65.2405 590.408,69.1923 
 596.063,73.2434 
 "/>
 
 
 
 
 u 
 
 
 net 
 
 
 
 u 
 
 
 ana